# House Sales in King County, USA
### Predict house price using regression

W tym zadaniu będziemy szacować ceny domów.

Dane zawierają takie dane jak:
- id - unikalny id domu
- date - data sprzedaży domu
- price - cena sprzedanego domu


- bedrooms
- bathrooms - liczba łazienek, gdzie 0.5 znaczy łazienka bez prysznica
- sqft_living - powierzchnia mieszkalna
- sqft_lot - powierzchnia ogródka
- sqft_above - powierzchnia domu nad powierzchnią ziemi 
- sqft_basement
- yr_built
- yr_renovated
- floors - liczba pięter


- waterfront - Zmienna określająca czy mieszkanie wychodziło na nabrzeże
- view - ocena widoku [0, 4]
- condition - stan mieszkania [1, 5]
- grade - ocena zgodności budynku z projektem i jakości wykonania [1, 13]


- zipcode
- lat - Szerokość geograficzna
- long - Długość geograficzna
- sqft_living15 - powierzchnia mieszkań najbliższych 15 sąsiadów
- sqft_lot15 - powierzchnia ziemi najbliższych sąsiadów

Wynika stąd, że dane typu "id" powinny zostać usunięte z tego względu, że mogą powodować szum, z drugiej strony w id może być ukryta jakaś informacja, z której model mógłby wywnioskować cenę. Aby przygotować dane należy też udostępnić datę sprzedaży w bardziej przystępnym formacie.

## 3. Dekompozycja Ceteris paribus dla wybranych obserwacji

<a href="./_plot_files/plots0.html" target="_top">Wykres</a>

<img src="./3.png">

## 4. Znalezienie dwóch obserwacji, gdzie dla jednej cena spada a dla drugiej stoi w miejscu przy zmianie roku budowy

<img src="./4.png">

Widać tutaj przykład, gdzie dla drogiego domu cena wraz z rokiem budowy spada na przełomie lat 1940-2000, natomiast dla domu taniego stoi w miejscu niezalażnie od roku budowy budynku.

## 5. Znalezienie obserwacji dla której CP się różni w zależności od modelu (XGBoost i Random Forest)

<img src="./5.png">

Widać tutaj, że CP różni się gdy grade > 10

## 6. Wyjaśnienie wyników

4. W tym przypadku możemy zaobserwować, że rok budowy nie ma takiego wpływu na cenę budynku jeśli cena jest niska. Natomiast rok budynku jest istotny w przypadku, gdy budynek jest wart znacznie więcej. Wtedy wyższą cenę osiągają budynki zbudowane przed 1940 rokiem oraz budynki, które zostały zbudowane stosunkowo niedawno. 

5. Tutaj jest przypadek, gdzie cena jest równa dla tych dwóch modeli, jeśli grade <= 10. Natomiast mocno od siebie odbiega w przypadku, gdy grade > 10. Może wynikać to z faktu, że inną zmienną skorelowaną z grade jest condition. Grade opisuje ocenę zgodności budynku z projektem i jakość wykonania [1, 13]. Natomiast condition opisuje stan mieszkania [1, 5]. Z opisu wynika, że te dwie zmienne opisują coś innego, lecz są one wystawiane przez człowieka i nie ma zbiorze danych przypadków gdzie grade jest oceniony wysoko np. 12 lub 13, a condition = 1 lub 2. Prawdopodobnie z takiego powodu te dwa modele mają inne predykcje dla takiego samego budynku przy zmieniającej się ocenie jakości wykonania.

# Załącznik

In [1]:
import numpy as np 
import pandas as pd 
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE 
import sklearn
from sklearn.ensemble import RandomForestRegressor
from ceteris_paribus.profiles import individual_variable_profile
from ceteris_paribus.explainer import explain
from ceteris_paribus.plots.plots import plot

In [2]:
df = pd.read_csv('kc_house_data.csv')
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


Dataframe zawiera dane typu :
 - 

Dane zawierają takie dane jak:
- id - unikalny id domu
- date - data sprzedaży domu
- price - cena sprzedanego domu


- bedrooms
- bathrooms - liczba łazienek, gdzie 0.5 znaczy łazienka bez prysznica
- sqft_living - powierzchnia mieszkalna
- sqft_lot - powierzchnia ogródka
- sqft_above - powierzchnia domu nad powierzchnią ziemi 
- sqft_basement
- yr_built
- yr_renovated
- floors - liczba pięter


- waterfront - Zmienna określająca czy mieszkanie wychodziło na nabrzeże
- view - ocena widoku [0, 4]
- condition - stan mieszkania [1, 5]
- grade - ocena zgodności budynku z projektem i jakości wykonania [1, 13]


- zipcode
- lat - Szerokość geograficzna
- long - Długość geograficzna
- sqft_living15 - powierzchnia mieszkań najbliższych 15 sąsiadów
- sqft_lot15 - powierzchnia ziemi najbliższych sąsiadów

Możemy wyszczególnić w tych danych 4 kategorie:
 - dane ogólne
 - specyfikacja domu
 - dane które są wyznaczane przez ludzi
 - dane o okolicy domu

Wynika stąd, że dane typu "id" powinny zostać usunięte z tego względu, że mogą powodować szum, z drugiej strony w id może być ukryta jakaś informacja, z której model mógłby wywnioskować cenę. Aby przygotować dane należy też udostępnić datę sprzedaży w bardziej przystępnym formacie.

In [3]:
df[['year', 'month', 'day']] = pd.DataFrame([ [int(x[0:4]), int(x[4:6]), int(x[6:8])] for x in df['date'].tolist() ])
df = df.drop(['date', 'id'], axis=1)
Y = df['price']
X = df.drop(['price'], axis=1)

In [4]:
df.groupby(['condition']).count()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,grade,sqft_above,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,day
condition,,,,,,,,,,,,,,,,,,,,,
1,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
2,172,172,172,172,172,172,172,172,172,172,...,172,172,172,172,172,172,172,172,172,172
3,14031,14031,14031,14031,14031,14031,14031,14031,14031,14031,...,14031,14031,14031,14031,14031,14031,14031,14031,14031,14031
4,5679,5679,5679,5679,5679,5679,5679,5679,5679,5679,...,5679,5679,5679,5679,5679,5679,5679,5679,5679,5679
5,1701,1701,1701,1701,1701,1701,1701,1701,1701,1701,...,1701,1701,1701,1701,1701,1701,1701,1701,1701,1701


In [5]:
#Xnp = X.to_numpy()
#Ynp = Y.to_numpy()

In [6]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y,  
                      test_size = 0.3, random_state = 123) 

### 1. Wytrenowany model

In [7]:
xgb_reg = XGBRegressor(colsample_bytree=0.7, learning_rate=0.06, max_depth=5, min_child_weight=6, n_estimators=700, nthread=1, objective='reg:squarederror', subsample=0.7)
xgb_reg.fit(train_X, train_Y)
score = xgb_reg.score(train_X, train_Y)  
print("Training score: ", score)
score = xgb_reg.score(test_X, test_Y)  
print("Test score: ", score)
pred = xgb_reg.predict(test_X)
rmse = np.sqrt(MSE(test_Y, pred)) 
print("RMSE : % f" %(rmse))

Training score:  0.9711698297228819
Test score:  0.905722369519171
RMSE :  113149.749407


## 2. Predykcja dla wybranych obserwacji

In [8]:
test_Y[:2]

5506    532500.0
9279    410000.0
Name: price, dtype: float64

In [9]:
xgb_reg.predict(test_X[:2])

array([617686.7 , 525400.25], dtype=float32)

In [10]:
list(X.columns)

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'year',
 'month',
 'day']

## 3. Dekompozycja Ceteris paribus dla wybranych obserwacji

In [11]:
house_cp_xgb = []
for i in range(2):
    explainer_xgb = explain(xgb_reg, data=X, y=Y, label=('XGBoost_house_'+str(i)),
    predict_function=lambda row: xgb_reg.predict(row))
    house = test_X.iloc[i]
    label_house = test_Y.iloc[i]
    house_cp_xgb.append(individual_variable_profile(explainer_xgb, house, label_house))
plot(*house_cp_xgb, selected_variables=["sqft_living"])

## 4. Znalezienie dwóch obserwacji, gdzie dla jednej cena opada a dla drugiej stoi w miejscu przy zmianie roku budowy

In [12]:
house_cp_xgb = []
for i in [13, 16]:
    explainer_xgb = explain(xgb_reg, data=X, y=Y, label=('XGBoost'+str(i)),
    predict_function=lambda row: xgb_reg.predict(row))
    house = test_X.iloc[i]
    label_house = test_Y.iloc[i]
    house_cp_xgb.append(individual_variable_profile(explainer_xgb, house, label_house))
plot(*house_cp_xgb, selected_variables=["yr_built"])

dla drogich domów cena wraz z rokiem budowy spada na przełomie lat 1940-2005, natomiast dla domów tanich stoi w miejscu

## 5. Wytrenowanie modelu random forest i znalezienie obserwacji dla której CP się różni

In [24]:
rndForest = RandomForestRegressor(random_state=0, n_estimators=1000)
rndForest.fit(train_X, train_Y)
score = rndForest.score(train_X, train_Y)  
print("Training score: ", score)
score = rndForest.score(test_X, test_Y)  
print("Test score: ", score)
pred = rndForest.predict(test_X)
rmse = np.sqrt(MSE(test_Y, pred)) 
print("RMSE : % f" %(rmse))

Training score:  0.9828493662369712
Test score:  0.8807256473820063
RMSE :  127269.049649


In [26]:
explainer_rf = explain(rndForest, data=X, y=Y, label='RandomForest',
    predict_function=lambda X: rndForest.predict(X))
explainer_xgb = explain(xgb_reg, data=X, y=Y, label=('XGBoost'),
    predict_function=lambda X: xgb_reg.predict(X))

k=11
observation = test_X.iloc[k]
label_observation = test_Y.iloc[k]
cp_rf = individual_variable_profile(explainer_rf, observation, label_observation)
cp_xgb = individual_variable_profile(explainer_xgb, observation, label_observation)
plot(cp_xgb, cp_rf, selected_variables=["grade"])

In [44]:
df13grade = df[(df['grade'] == 13)]
df13grade
df12grade = df[(df['grade'] == 12)]
df12grade

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,day
153,2250000.0,4,3.25,5180,19850,2.0,0,3,3,12,...,2006,0,98006,47.5620,-122.162,3160,9750,2015,4,1
419,1550000.0,5,4.25,6070,171626,2.0,0,0,3,12,...,1999,0,98024,47.5954,-121.950,4680,211267,2014,7,10
540,2125000.0,3,2.50,5403,24069,2.0,1,4,4,12,...,1976,0,98166,47.4169,-122.348,3980,104374,2015,2,18
722,1578000.0,4,3.25,4670,51836,2.0,0,0,4,12,...,1988,0,98005,47.6350,-122.164,4230,41075,2014,8,8
799,1100000.0,4,3.50,4270,40097,1.0,0,0,4,12,...,1993,0,98077,47.7354,-122.078,3510,36149,2015,2,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20441,2998000.0,5,4.00,6670,16481,2.0,0,0,3,12,...,2007,0,98004,47.6409,-122.221,4800,16607,2015,5,7
20471,1800000.0,4,3.50,6370,205603,2.0,0,0,3,12,...,2008,0,98027,47.5016,-121.905,1490,33580,2015,1,23
21050,900000.0,5,6.00,7120,40806,2.0,0,4,3,12,...,2007,0,98006,47.5451,-122.114,3440,36859,2015,2,19
21344,1488000.0,5,6.00,6880,279968,2.0,0,3,3,12,...,2007,0,98045,47.4624,-121.779,4690,256803,2014,8,22


In [47]:
df13grade1cond = df12grade[(df12grade['condition'] == 3)]
df13grade1cond

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,day
153,2250000.0,4,3.25,5180,19850,2.0,0,3,3,12,...,2006,0,98006,47.5620,-122.162,3160,9750,2015,4,1
419,1550000.0,5,4.25,6070,171626,2.0,0,0,3,12,...,1999,0,98024,47.5954,-121.950,4680,211267,2014,7,10
1164,5110800.0,5,5.25,8010,45517,2.0,1,4,3,12,...,1999,0,98033,47.6767,-122.211,3430,26788,2014,10,20
1294,835000.0,4,4.25,4930,25714,2.0,0,0,3,12,...,2005,0,98092,47.3069,-122.148,3620,23035,2014,7,3
1448,5350000.0,5,5.00,8000,23985,2.0,0,4,3,12,...,2009,0,98004,47.6232,-122.220,4600,21750,2015,4,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20441,2998000.0,5,4.00,6670,16481,2.0,0,0,3,12,...,2007,0,98004,47.6409,-122.221,4800,16607,2015,5,7
20471,1800000.0,4,3.50,6370,205603,2.0,0,0,3,12,...,2008,0,98027,47.5016,-121.905,1490,33580,2015,1,23
21050,900000.0,5,6.00,7120,40806,2.0,0,4,3,12,...,2007,0,98006,47.5451,-122.114,3440,36859,2015,2,19
21344,1488000.0,5,6.00,6880,279968,2.0,0,3,3,12,...,2007,0,98045,47.4624,-121.779,4690,256803,2014,8,22
